In [1]:
from struct import pack


# byte를 int로 변경 
def bytes_to_int(num):  #convert bytes object to an int
    res = 0
    num = num[::-1]  # reverse the bytes
    for i in range(len(num)):
        res += num[i] * (256**i)
    return res

# unpack
def get_fields(f):
    chunk = bytes_to_int(f.read(4))  # read 2 bytes, f1-f3, convert to int
    print(chunk)
    f1 = (chunk & 31) >> 0x80000000  # get each field with its bitmask
    print(f1)

    return f1

def read_file():
    file = open('data/eventcg.pak','rb')

    #using your sample data
    print(get_fields(file))  # returns 0, 4, 384, field4 as a bytes obj
    
    file.close()
 

class Bitmap():
    def __init__(s, width, height):
        s._bfType = 19778 # Bitmap signature
        s._bfReserved1 = 0
        s._bfReserved2 = 0
        s._bcPlanes = 1
        s._bcSize = 12
        s._bcBitCount = 24
        s._bfOffBits = 26
        s._bcWidth = width
        s._bcHeight = height
        s._bfSize = 26+s._bcWidth*3*s._bcHeight
        s.clear()
        
    def clear(s):
        s._graphics = [(0,0,0)]*s._bcWidth*s._bcHeight
        
        
    def setPixel(s, x, y, color):
        if isinstance(color, tuple):
            if x < 0 or y < 0 or x > s._bcWidth - 1 or y > s._bcHeight - 1:
                raise ValueError('Coords out of range')
                
            if len(color) != 3:
                raise ValueError('Color must be a tuple of 3 elems')
            s._graphics[y*s._bcWidth+x] = (color[2], color[1], color[0])
        else:
            raise ValueError('Color must be a tuple of 3 elems')


    def write(s, file):
        with open(file, 'wb') as f:
            f.write(pack('<HLHHL', 
                   s._bfType, 
                   s._bfSize, 
                   s._bfReserved1, 
                   s._bfReserved2, 
                   s._bfOffBits)) # Writing BITMAPFILEHEADER
            f.write(pack('<LHHHH', 
                   s._bcSize, 
                   s._bcWidth, 
                   s._bcHeight, 
                   s._bcPlanes, 
                   s._bcBitCount)) # Writing BITMAPINFO
            for px in s._graphics:
                f.write(pack('<BBB', *px))
            for i in range((4 - ((s._bcWidth*3) % 4)) % 4):
                f.write(pack('B', 0))



def main():
    
    read_file()

#    side = 512
#     b = Bitmap(side, side)
#     for j in range(0, side):
#         b.setPixel(j, j, (255, 0, 0))
#         b.setPixel(j, side-j-1, (255, 0, 0))
#         b.setPixel(j, 0, (255, 0, 0))
#         b.setPixel(j, side-1, (255, 0, 0))
#         b.setPixel(0, j, (255, 0, 0))
#         b.setPixel(side-1, j, (255, 255, 255))
#     b.write('data/file.bmp')


if __name__ == '__main__':
    main()


FileNotFoundError: [Errno 2] No such file or directory: 'data/eventcg.pak'